<a href="https://colab.research.google.com/github/mahsahadian/Cloud-TP2/blob/master/Final/SVM_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import SVR
from sklearn import metrics
from sklearn.metrics import accuracy_score
import pickle


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
url = 'https://raw.githubusercontent.com/tna-hub/Cloud-TP2/master/dataset.csv'
data = pd.read_csv(url)
data.head()


,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,6.0,18.0,5.0,NaN,800.0,88.0,0.1,1021.1,-18.6,0.0,NW,4.4,Gucheng
1,2,2013,3,1,1,6.0,15.0,5.0,NaN,800.0,88.0,-0.3,1021.5,-19.0,0.0,NW,4.0,Gucheng
2,3,2013,3,1,2,5.0,18.0,NaN,NaN,700.0,52.0,-0.7,1021.5,-19.8,0.0,WNW,4.6,Gucheng
3,4,2013,3,1,3,6.0,20.0,6.0,NaN,NaN,NaN,-1.0,1022.7,-21.2,0.0,W,2.8,Gucheng
4,5,2013,3,1,4,5.0,17.0,5.0,NaN,600.0,73.0,-1.3,1023.0,-21.4,0.0,WNW,3.6,Gucheng


In [3]:
data.isnull().sum()  # showing the number of rows with null values


No             0
year           0
month          0
day            0
hour           0
PM2.5       8043
PM10        5965
SO2         8352
NO2        11362
CO         19404
O3         12199
TEMP         379
PRES         374
DEWP         384
RAIN         371
wd          1743
WSPM         305
station        0
dtype: int64

In [4]:
data = data.dropna()  # drop all rows with null values
data.isnull().sum()  # verify it worked 

No         0
year       0
month      0
day        0
hour       0
PM2.5      0
PM10       0
SO2        0
NO2        0
CO         0
O3         0
TEMP       0
PRES       0
DEWP       0
RAIN       0
wd         0
WSPM       0
station    0
dtype: int64

In [5]:
data.describe()  # get more info about data

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM
count,349400.000000,349400.000000,349400.000000,349400.000000,349400.000000,349400.000000,349400.000000,349400.000000,349400.000000,349400.000000,349400.000000,349400.000000,349400.000000,349400.000000,349400.000000,349400.000000
mean,17969.317490,2014.716657,6.472350,15.711829,11.579379,78.981647,103.885010,15.492473,50.125805,1217.505690,57.513042,13.494827,1010.743455,2.399353,0.065279,1.736347
std,9998.752658,1.159938,3.449074,8.803836,6.934095,79.712207,90.840935,21.055879,34.969247,1150.170175,56.676463,11.430139,10.440024,13.805684,0.827349,1.244607
min,1.000000,2013.000000,1.000000,1.000000,0.000000,2.000000,2.000000,0.285600,2.000000,100.000000,0.214200,-19.900000,982.400000,-36.000000,0.000000,0.000000
25%,9629.000000,2014.000000,3.000000,8.000000,6.000000,20.000000,36.000000,2.000000,23.000000,500.000000,11.000000,3.100000,1002.300000,-9.000000,0.000000,0.900000
50%,18120.000000,2015.000000,6.000000,16.000000,12.000000,54.000000,81.000000,7.000000,43.000000,900.000000,45.000000,14.400000,1010.300000,2.900000,0.000000,1.400000
75%,26522.000000,2016.000000,9.000000,23.000000,18.000000,110.000000,145.000000,19.000000,70.000000,1500.000000,82.000000,23.200000,1019.000000,15.100000,0.000000,2.200000
max,35064.000000,2017.000000,12.000000,31.000000,23.000000,844.000000,999.000000,500.000000,290.000000,10000.000000,1071.000000,41.600000,1042.800000,29.100000,72.500000,12.900000


In [0]:
labelencoder = LabelEncoder()
data['wd'] = labelencoder.fit_transform(data['wd'])
data['station'] = labelencoder.fit_transform(data['station'])


In [0]:
data

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
20,21,2013,3,1,20,13.0,25.0,12.0,5.0,1100.0,61.0,1.6,1027.1,-18.4,0.0,2,1.9,5
21,22,2013,3,1,21,15.0,23.0,14.0,13.0,1200.0,52.0,1.0,1028.1,-17.4,0.0,10,0.7,5
22,23,2013,3,1,22,16.0,28.0,16.0,19.0,1200.0,45.0,1.3,1028.4,-17.6,0.0,0,1.0,5
23,24,2013,3,1,23,16.0,28.0,14.0,20.0,1100.0,44.0,0.2,1028.6,-17.6,0.0,2,1.1,5
24,25,2013,3,2,0,14.0,18.0,24.0,43.0,1399.0,25.0,-0.3,1028.9,-18.0,0.0,1,1.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385699,35060,2017,2,28,19,11.0,27.0,4.0,20.0,300.0,81.0,12.6,1011.9,-14.3,0.0,3,2.0,10
385700,35061,2017,2,28,20,15.0,43.0,6.0,55.0,500.0,45.0,9.4,1012.3,-11.9,0.0,15,1.0,10
385701,35062,2017,2,28,21,13.0,35.0,7.0,48.0,500.0,48.0,8.7,1012.8,-13.7,0.0,3,1.1,10
385702,35063,2017,2,28,22,12.0,31.0,5.0,47.0,500.0,50.0,7.8,1012.9,-12.6,0.0,5,1.0,10


In [7]:
data.corr()  # check for highest correlation values

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
No,1.000000,0.968604,0.056135,0.017161,-0.003799,-0.029323,-0.049093,-0.226194,-0.043095,0.028447,-0.049297,-0.134103,0.186158,-0.111080,0.005317,-0.021251,0.055228,-0.005576
year,0.968604,1.000000,-0.192972,-0.008328,-0.003975,-0.032162,-0.043294,-0.166929,-0.052502,0.016569,-0.023017,-0.168096,0.188252,-0.179834,0.000920,-0.012707,0.090281,-0.006688
month,0.056135,-0.192972,1.000000,0.018011,-0.001608,0.012734,-0.022081,-0.224337,0.039098,0.046540,-0.101499,0.147863,-0.025851,0.285297,0.017748,-0.033399,-0.144272,0.004666
day,0.017161,-0.008328,0.018011,1.000000,-0.002192,0.003604,0.023292,-0.007298,0.012910,-0.018428,-0.000920,0.011928,0.023196,0.023730,-0.002491,0.002133,-0.013400,0.001787
hour,-0.003799,-0.003975,-0.001608,-0.002192,1.000000,0.013606,0.051150,0.033588,0.000951,-0.029654,0.280606,0.139736,-0.038567,-0.012335,0.013119,0.164512,0.137427,0.005675
PM2.5,-0.029323,-0.032162,0.012734,0.003604,0.013606,1.000000,0.884017,0.479773,0.667725,0.789387,-0.146996,-0.124145,0.012171,0.121528,-0.014920,-0.079139,-0.273450,0.026361
PM10,-0.049093,-0.043294,-0.022081,0.023292,0.051150,0.884017,1.000000,0.461051,0.650845,0.699844,-0.111283,-0.090685,-0.023393,0.075048,-0.026542,-0.039990,-0.181472,0.022254
SO2,-0.226194,-0.166929,-0.224337,-0.007298,0.033588,0.479773,0.461051,1.000000,0.501781,0.532080,-0.164773,-0.318438,0.221443,-0.264875,-0.040567,-0.018950,-0.106464,0.004933
NO2,-0.043095,-0.052502,0.039098,0.012910,0.000951,0.667725,0.650845,0.501781,1.000000,0.701348,-0.472250,-0.274383,0.175303,-0.029074,-0.043883,-0.118394,-0.396123,0.081273
CO,0.028447,0.016569,0.046540,-0.018428,-0.029654,0.789387,0.699844,0.532080,0.701348,1.000000,-0.313439,-0.323086,0.185726,-0.055313,-0.013169,-0.126399,-0.294312,0.035658


In [0]:
#  split the data into training and test dataset - 80 % - train, 20 % - test

In [9]:
columns = ["year", "month", "hour", "PM2.5", "PM10", "SO2", "NO2", "CO", "O3", "PRES", "DEWP", "wd"]

norm_mean_dict = {}
norm_stdev_dict = {}

for column in columns:
    norm_stdev_dict.update( {column : data[column].std() } )
    norm_mean_dict.update( {column : data[column].mean() } )
    data[column] = (data[column] - data[column].mean()) / data[column].std() + 3


float

In [0]:
norm_mean_dict

{'CO': 1217,
 'DEWP': 2,
 'NO2': 50,
 'O3': 57,
 'PM10': 103,
 'PM2.5': 78,
 'PRES': 1010,
 'SO2': 15,
 'hour': 11,
 'month': 6,
 'wd': 6,
 'year': 2014}

In [0]:
norm_stdev_dict

{'CO': 1150,
 'DEWP': 13,
 'NO2': 34,
 'O3': 56,
 'PM10': 90,
 'PM2.5': 79,
 'PRES': 10,
 'SO2': 21,
 'hour': 6,
 'month': 3,
 'wd': 4,
 'year': 1}

In [0]:
data

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
20,21,1.520044,1.993252,1,4.214379,2.172252,2.131614,2.834133,1.709557,2.897836,3.061524,1.6,4.566715,1.493421,0.0,1.956500,1.9,5
21,22,1.520044,1.993252,1,4.358594,2.197342,2.109597,2.929118,1.938330,2.984780,2.902728,1.0,4.662500,1.565855,0.0,3.724852,0.7,5
22,23,1.520044,1.993252,1,4.502809,2.209887,2.164639,3.024104,2.109909,2.984780,2.779220,1.3,4.691236,1.551368,0.0,1.514412,1.0,5
23,24,1.520044,1.993252,1,4.647024,2.209887,2.164639,2.929118,2.138506,2.897836,2.761576,0.2,4.710393,1.551368,0.0,1.956500,1.1,5
24,25,1.520044,1.993252,2,1.330081,2.184797,2.054556,3.404045,2.796227,3.157798,2.426340,-0.3,4.739129,1.522395,0.0,1.735456,1.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385699,35060,4.968504,1.703319,28,4.070164,2.147161,2.153630,2.454192,2.138506,2.202287,3.414404,12.6,3.110780,1.790400,0.0,2.177544,2.0,10
385700,35061,4.968504,1.703319,28,4.214379,2.197342,2.329762,2.549177,3.139385,2.376174,2.779220,9.4,3.149094,1.964242,0.0,4.830072,1.0,10
385701,35062,4.968504,1.703319,28,4.358594,2.172252,2.241696,2.596670,2.939209,2.376174,2.832152,8.7,3.196987,1.833861,0.0,2.177544,1.1,10
385702,35063,4.968504,1.703319,28,4.502809,2.159707,2.197663,2.501684,2.910613,2.376174,2.867440,7.8,3.206565,1.913538,0.0,2.619632,1.0,10


In [0]:
#x = data[["year", "month", "day", "hour", "PM2.5", "PM10", "SO2", "NO2", "CO", "O3", "PRES", "DEWP", "RAIN", "wd", "WSPM", "station"]]
x = data[["year", "month", "hour", "PM2.5", "PM10", "SO2", "NO2", "CO", "O3", "PRES", "DEWP", "wd"]]
y = data["TEMP"]
train_data = pd.DataFrame()
test_data = pd.DataFrame()
train_data, test_data, train_label, test_label = train_test_split(x, y, test_size=0.2)



In [0]:
# Encode sequentially the string values

In [0]:
#labelencoder = LabelEncoder()
#train_data['wd'] = labelencoder.fit_transform(train_data['wd'])
#train_data['station'] = labelencoder.fit_transform(train_data['station'])
#test_data['wd'] = labelencoder.fit_transform(test_data['wd'])
#test_data['station'] = labelencoder.fit_transform(test_data['station'])


In [0]:
train_data  # verify it worked

,year,month,hour,PM2.5,PM10,SO2,NO2,CO,O3,PRES,DEWP,wd
186037,2.382159,2.573117,3.204875,3.251133,3.760835,3.546523,3.711316,3.506442,4.702417,1.827258,3.934445,3.945896
61818,4.106389,1.993252,3.925949,2.623876,3.452604,2.596670,2.424471,2.463118,3.396760,3.608863,2.529226,3.724852
81333,2.382159,2.863050,4.358594,3.150772,2.858159,2.501684,2.767630,3.506442,3.555556,2.114614,3.992392,1.514412
63660,4.106389,2.863050,3.060660,3.075501,2.792109,3.214074,2.081313,2.723949,5.708125,2.143349,3.731630,3.945896
65297,4.106389,3.442916,3.781734,3.263678,2.957233,2.501684,2.138506,2.897836,5.302313,2.095457,4.709488,3.945896
...,...,...,...,...,...,...,...,...,...,...,...,...
71433,1.520044,2.283184,2.628015,4.480556,4.212174,2.929118,3.768509,3.071724,2.196967,2.124192,3.369460,3.724852
82953,2.382159,3.442916,2.628015,2.448245,2.307746,2.454192,2.109909,2.463118,4.014300,2.057142,4.028609,4.830072
234273,3.244274,4.312715,2.628015,2.623876,2.395812,2.359206,2.510261,4.897540,2.020527,4.135682,2.818962,1.514412
2903,1.520044,2.863050,4.647024,4.254743,3.760835,2.406699,4.254651,4.288935,2.055815,1.760208,4.426995,2.177544


In [0]:
test_data  

,year,month,hour,PM2.5,PM10,SO2,NO2,CO,O3,PRES,DEWP,wd
172795,4.106389,4.312715,4.070164,3.012775,3.342522,3.309060,4.168861,3.941160,2.020527,3.752541,2.674094,1.735456
344933,4.106389,3.152983,2.051155,2.623876,2.604969,2.501684,3.310965,2.637005,2.585136,2.708482,4.057582,4.387984
131857,4.106389,1.993252,1.474296,4.317469,4.509397,3.356553,4.025878,3.506442,2.073459,2.679746,2.500253,3.724852
146083,1.520044,4.022782,4.070164,2.410609,2.472870,2.691655,3.968685,2.637005,2.126391,4.317674,2.761015,4.609028
95192,4.106389,1.413386,2.483800,3.075501,3.034291,3.641509,3.654123,4.897540,2.038171,3.532235,1.710723,2.177544
...,...,...,...,...,...,...,...,...,...,...,...,...
151411,2.382159,2.863050,4.070164,2.209887,2.351779,2.359206,3.396754,2.637005,2.973304,1.721894,3.898228,1.514412
100630,4.106389,3.442916,4.502809,2.385519,2.693035,2.359206,2.796227,2.637005,3.696708,2.066721,4.332831,2.840676
378345,4.106389,2.283184,2.628015,3.276223,3.232439,2.596670,3.797106,2.810893,2.549848,3.034152,3.717143,1.514412
97520,4.106389,2.283184,2.483800,2.435699,2.671018,2.786640,3.396754,2.723949,2.073459,2.459440,2.811719,3.061720


In [0]:
# normilize train and test data using z-score and shift it to avoid negative numbers
# use the same normalization pararameters for train and test datasets
# P.S. if u know how to avoid the warnings tell me please

In [0]:
# split training data into training (75%) and validation (25%) datasets

In [0]:
x = train_data[["year", "month", "hour", "PM2.5", "PM10", "SO2", "NO2", "CO", "O3", "PRES", "DEWP", "wd"]]
y = train_label
validation_data = pd.DataFrame()
train_data, validation_data, train_label, validation_label = train_test_split(x, y, test_size=0.25)

In [0]:
train_data

,year,month,hour,PM2.5,PM10,SO2,NO2,CO,O3,PRES,DEWP,wd
98812,4.106389,2.863050,1.906940,3.025320,2.748076,2.359206,2.167102,2.463118,2.391052,1.970936,3.992392,2.840676
207212,4.106389,4.022782,4.214379,5.308534,4.905694,2.549177,4.769389,4.288935,2.020527,3.216144,3.876498,2.840676
99217,4.106389,2.863050,1.474296,2.071891,1.911449,2.359206,1.909733,2.376174,3.273252,1.817679,3.224592,2.398588
184474,2.382159,1.993252,2.772230,4.681278,10.817126,4.876318,4.426230,3.854216,2.020527,2.574383,2.449549,2.840676
153971,2.382159,3.732849,2.916445,2.536060,2.307746,2.359206,2.939209,2.984780,2.179323,3.311929,4.006879,2.619632
...,...,...,...,...,...,...,...,...,...,...,...,...
31469,4.106389,4.022782,2.051155,4.806729,4.311248,2.359206,2.767630,3.332555,2.726288,2.641432,3.731630,2.840676
57194,3.244274,3.732849,1.618511,2.862234,3.012274,2.406699,5.341320,2.897836,2.214612,3.187408,4.108286,1.735456
317113,1.520044,2.573117,1.474296,2.924959,2.759084,3.024104,2.052716,2.723949,3.837860,3.216144,3.231835,4.166940
84067,2.382159,4.022782,4.070164,2.247522,2.065564,2.406699,3.282368,2.463118,2.161679,2.526490,3.441894,1.514412


In [0]:
validation_data

,year,month,hour,PM2.5,PM10,SO2,NO2,CO,O3,PRES,DEWP,wd
114370,2.382159,1.993252,2.772230,2.109526,2.197663,2.406699,1.652364,2.115343,3.502624,3.359821,2.275708,3.282764
246912,1.520044,2.573117,1.330081,2.372974,3.771844,2.691655,2.624647,2.376174,3.379116,3.302350,2.746528,1.514412
294446,2.382159,4.022782,3.349090,2.586241,2.362787,2.454192,2.567454,2.637005,2.832152,2.765953,3.673682,1.956500
349687,4.968504,1.413386,2.339585,2.096981,2.109597,2.406699,2.567454,2.550062,2.743932,5.036063,1.210929,3.061720
22115,3.244274,3.732849,2.916445,2.761873,2.715051,2.501684,2.653244,2.463118,4.014300,2.698903,3.985148,1.956500
...,...,...,...,...,...,...,...,...,...,...,...,...
115335,2.382159,2.283184,3.493305,2.937505,2.428837,2.359206,1.795347,2.115343,4.631841,2.794689,2.695824,3.503808
4735,1.520044,3.732849,2.339585,2.799508,2.660010,2.311714,3.196578,3.245611,2.002883,2.076299,4.238667,2.177544
349859,4.968504,1.413386,2.916445,2.473335,2.461861,3.071596,2.138506,2.637005,3.132100,4.767864,1.703479,4.166940
209110,4.968504,1.413386,4.502809,4.606007,4.135116,3.309060,5.226934,2.289231,2.020527,3.982425,2.616147,1.735456


In [0]:
clf = svm.SVR() # create the regression svm model

In [0]:
clf.fit(train_data, train_label) #  train the model

In [0]:
filename = '/content/drive/My Drive/Colab Notebooks/svm_model_2.0.sav'
#pickle.dump(clf, open(filename, 'wb'))

with open('/content/drive/My Drive/Colab Notebooks/svm_model_2.0.sav', 'wb') as f:
    pickle.dump(clf, f)


In [0]:
loaded_model = pickle.load(open(filename, 'rb'))  # open the saved model

In [0]:
accuracy = loaded_model.score(validation_data, validation_label)  # check accuracy on validation dataset
print(accuracy)

In [0]:
accuracy = loaded_model.score(test_data, test_label)  # check accuracy on test dataset
print(accuracy)

In [0]:
temp_predicted = loaded_model.predict(test_data)  # calculate predictions for test dataset

In [0]:
df_test = pd.DataFrame({'Actual': test_label, 'Predicted': temp_predicted})  # create dataset with predictions

In [0]:
df_test

In [0]:
#  add categorization labels

In [0]:
group_names = ['verycold', 'cold', 'moderate', 'hot', 'veryhot']
bins = [-30, 0, 10, 20, 30, 50]
df_test["Actual_category"] = pd.cut(df_test["Actual"], bins = bins, include_lowest=True, labels = group_names)
df_test["Predicted_category"] = pd.cut(df_test["Predicted"], bins = bins, include_lowest=True, labels = group_names)

In [0]:
df_test  # show the result